In [1]:
%load_ext tensorboard.notebook

In [2]:
!rm -rf ./logs/ 

In [3]:
import tensorflow as tf
import datetime

In [4]:
from tensorflow.keras import datasets, layers, models


In [5]:
class CNN():
    def __init__(self, img_width, img_height, channels):
        self.img_width = img_width
        self.image_height = img_height
        self.channels = channels
        self.model = None
        self.tb_callback = None
    
    def gen_model(self, dropout=0.2, depth=32):
        self.model = models.Sequential()
        
        self.model.add(layers.Conv2D(depth, (3, 3), activation='relu', input_shape=(28, 28, 1)))
        self.model.add(layers.MaxPooling2D((2, 2)))
        self.model.add(layers.Dropout(dropout))
        
        self.model.add(layers.Conv2D(depth*2, (3, 3), activation='relu'))
        self.model.add(layers.MaxPooling2D((2, 2)))
        self.model.add(layers.Dropout(dropout))
        
        self.model.add(layers.Conv2D(depth*2, (3, 3), activation='relu'))
        self.model.add(layers.Flatten())
        self.model.add(layers.Dense(depth*2, activation='relu'))
        self.model.add(layers.Dense(10, activation='softmax'))
        
        self.model.summary()
        
        return self.model
    
    def train(self,
              train_images, 
              train_labels,
              optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              tensorboard=True,
              epochs=10
              ):
        
        self.model.compile(optimizer=optimizer, loss=loss,  metrics=['accuracy'])
        
        if tensorboard:
            log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            self.tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
            self.model.fit(train_images, train_labels, epochs=epochs, callbacks=[self.tb_callback])
        else:
            self.model.fit(train_images, train_labels, epochs=epochs)
  
    
    def evaluate(self,test_images, test_labels):
        test_loss, test_acc = self.model.evaluate(test_images, test_labels)
        print("Loss : \n {}".format(test_loss))
        print("Accruacy : \n {}".format(test_acc))

        

In [6]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

In [7]:
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

In [8]:
cnn = CNN(28,28,1)
cnn.gen_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          3

In [9]:
cnn.train(train_images, train_labels)

Epoch 1/10
60000/60000 [==============================] - 13s 217us/sample - loss: 0.4241 - accuracy: 0.9051
Epoch 2/10
60000/60000 [==============================] - 14s 226us/sample - loss: 0.0896 - accuracy: 0.9727
Epoch 3/10
60000/60000 [==============================] - 14s 230us/sample - loss: 0.0719 - accuracy: 0.9781
Epoch 4/10
60000/60000 [==============================] - 14s 230us/sample - loss: 0.0647 - accuracy: 0.9802
Epoch 5/10
60000/60000 [==============================] - 14s 229us/sample - loss: 0.0563 - accuracy: 0.9836
Epoch 6/10
60000/60000 [==============================] - 13s 223us/sample - loss: 0.0528 - accuracy: 0.9846
Epoch 7/10
60000/60000 [==============================] - 13s 223us/sample - loss: 0.0483 - accuracy: 0.9857
Epoch 8/10
60000/60000 [==============================] - 13s 224us/sample - loss: 0.0437 - accuracy: 0.9871
Epoch 9/10
60000/60000 [==============================] - 13s 222us/sample - loss: 0.0440 - accuracy: 0.9870
Epoch 10/10
60000/6

In [10]:
%tensorboard --logdir logs/fit